# Interactive Map (Folium Module) to show electric vehicle charging stations in Copenhagen

In [6]:
import geopandas as gpd
import folium
from pyproj import CRS

#import matplotlib.pyplot as plt
#import contextily as ctx
from folium.plugins import MarkerCluster

%matplotlib inline


url5='https://wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:elbil_ladestander&outputFormat=json&SRSNAME=EPSG:4326'
elbil_ladning = gpd.read_file(url5)

In [4]:
# Reproject to EPSG 3857 (wed mercator projection), which matches raster tiles imported by contextily basemaps.

elbil_ladning = elbil_ladning.to_crs(epsg=3857)

In [30]:
# Explode Geodataframe to reduce multipart geometries (in this case multipoints to points)
explode = elbil_ladning.explode()
# Extract coordinates from geometries
locations = [(y,x) for y,x in zip(explode['geometry'].y , explode['geometry'].x)]
# Prepare popup text as list
pop = [x for x in explode['betalingszone']]
pop1 = [sub.replace('å', 'aa') for sub in pop] 
popups = [sub.replace('ø', 'oe') for sub in pop1]


# Extract lats and lons seperately
lats = [y for y in explode['geometry'].y]
lons = [x for x in explode['geometry'].x]

In [56]:

# Create a Map instance
m = folium.Map(location=[55.685811,12.581921], zoom_start=13, prefer_canvas=True, control_scale=True, tiles='Stamen Watercolor') #'Cartodb PositronStamen Watercolor'

#%%time

marker_cluster = MarkerCluster(
    name='Electric Vehicle Charging Stations',
    overlay=True,
    control=True,
    icon_create_function=None
)

for k in range(len(lats)):
    location = lats[k], lons[k]
    marker = folium.Marker(location=location, icon=folium.Icon(color='black', icon='plug', prefix='fa')) # Font Awesome Icons version 4, 5 not available in Folium yet.
    popup = f'Priszone: {popups[k]}'
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)
m

In [57]:
outfp = r"Electric.html"
m.save(outfp)

#help(folium.Marker)